# HACKATHON_Yandex.Music

### **Цель:** Разработка ML-модели для сопоставления текстов музыкальных произведений и для поиска каверов (вариации обработки оригинала с элементами новой аранжировки) по их текстам

### **Задача**: Разработать ML-продукт, который:

- Находит все кавер-треки и/или исходники к заданному треку в датасете
- Перечисляет все кавер-треки и/или оригиналы к заданному, указывает положение данного трека в цепочке каверов

### **Стек:** *pandas, pyplot, seaborn, sklearn, CatBoost, XGBoost, LightGBM, NLP*

### **Описание данных**

#### Разметка каверов:

Файл `covers.json` содержит разметку каверов, сделанную редакторами сервиса:

- `track_id` - уникальный идентификатор трека;
- `track_remake_type` - метка, присвоенная редакторами. Может принимать значения `ORIGINAL` и `COVER`;
- `original_track_id` - уникальный идентификатор исходного трека.

<aside>
💡 Обратите внимание, что не для всех каверов известны идентификаторы исходных треков!!!

</aside>

#### Метаинформация:

- `track_id` - уникальный идентификатор трека;
- `dttm` - первая дата появления информации о треке;
- `title` - название трека;
- `language` - язык исполнения;
- `isrc` - международный уникальный идентификатор трека;
- `genres` - жанры;
- `duration` - длительность трека;

#### Текст песен:

- `track_id` - уникальный идентификатор трека;
- `lyricId` - уникальный идентификатор текста;
- `text` - текст трека.


###**Описание решения:**
  1. Загрузка данных.
   - Подключение необходимых библиотек.
   - Предварительное изучение данных.
   - Предварительная подготовка данных. Отчистка от явно лишней информации.
   - Формирование общего датасета для дальнейшего исследование и предобработки.
  2. Исследовательский анализ:
    - Работа с аномалиями, пропусками и дубликатами.
    - Анализ признаков.
  3. Формирование признаков:
    - Анализ данных,
    - Удаление неинформативных признаков, генерация ряда признаков, по необходимости.
  4. Построение и обучение модели:
    - Подготовка данных для обучения модели:
      - Кодирование и масштабирование признаков - стандартизация данных, по необходимости.
      - Разделение общего датасета на выборки для обучения и проверки модели.
    - Классические модели:
       - Предварительная проверка моделей.
       - Линейная регрессия, Ridge.
       - CatBoost,
  5. Выбор лучшей модели. Тестирование.
  6. Выводы.

## **1.Загрузка данных.**
   - Подключение необходимых библиотек.

In [5]:
#проект запускаю в colab
import sys
ENV_COLAB = 'google.colab' in sys.modules

if ENV_COLAB:
    !pip install catboost >> None
    !"{sys.executable}" -m pip install phik >> None
    !pip install pytorch-transformers >> None
    !pip install transformers >> None
    #!pip install pytorch-pretrained-bert >> None
    !pip install pyinflect >> None
    !pip install sentence_splitter >> None
    !pip install contractions >> None
    !pip install sentence-transformers >>None
    !pip install langdetect >>None


    print('Environment: Google Colab')

import phik
from phik.report import plot_correlation_matrix
from phik import report, phik_matrix

# работа с текстом
import transformers as ppb
from langdetect import detect #определение языка
#https://pypi.org/project/pyinflect/
import pyinflect
from sentence_splitter import SentenceSplitter, split_text_into_sentences
#Fixes contractions such as `you're` to you `are`
import contractions

Environment: Google Colab


In [6]:
# импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
#from phik import phik_matrix

#from joblib import dump
import json
import math
import time
import re

# импорт моделей
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

# импортируем функции из statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from scipy.stats import norm
from scipy import stats

# предварительная обработка
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
# работа с текстом
import gensim
from gensim.utils import simple_preprocess
import spacy
import en_core_web_sm # малая модель spacy
import gensim.downloader as api
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

# кроссвалидация
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, TimeSeriesSplit, cross_val_score

# метрики
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, roc_curve

# настройки
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
#from skimpy import clean_columns
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

# константа верхний регистр
RANDOM_STATE = 123456

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


### **Загрузка данных**
   - Предварительное изучение данных.
   - Предварительная подготовка данных. Отчистка от явно лишней информации.
   - Формирование общего датасета для дальнейшего исследование и предобработки.Импорты необходимых билиотек

In [9]:
def convert_js(data):
    data_js = []
    with open(f'{data}', 'r') as file:
        for line in file:
            try:
                item = json.loads(line)
                # Преобразовать значения списков в строки
                for key, value in item.items():
                    if isinstance(value, list):
                        item[key] = json.dumps(value)
                data_js.append(item)
            except json.JSONDecodeError as e:
                print(f"Ошибка при разборе JSON: {e}")
    return data_js

In [10]:
covers_js = convert_js('covers.json')
lyrics_js = convert_js( 'lyrics.json')
meta_js = convert_js( 'meta.json')

In [11]:
# Создаем DataFrame из данных JSON#
covers = pd.DataFrame(covers_js)
covers.head()

,original_track_id,track_id,track_remake_type
0,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL
1,fe7ee8fc1959cc7214fa21c4840dff0a,fe7ee8fc1959cc7214fa21c4840dff0a,ORIGINAL
2,cd89fef7ffdd490db800357f47722b20,cd89fef7ffdd490db800357f47722b20,ORIGINAL
3,995665640dc319973d3173a74a03860c,995665640dc319973d3173a74a03860c,ORIGINAL
4,None,d6288499d0083cc34e60a077b7c4b3e1,COVER


In [12]:
lyrics = pd.DataFrame(lyrics_js)
lyrics.head()

,lyricId,text,track_id
0,a951f9504e89759e9d23039b7b17ec14,"Живу сейчас обломами, обломками не той любви\n...",1c4b1230f937e4c548ff732523214dcd
1,0c749bc3f01eb8e6cf986fa14ccfc585,Tell me your fable\nA fable\nTell me your fabl...,0faea89b0d7d6235b5b74def72511bd8
2,e2c8830fbc86e5964478243099eec23a,You're ashamed about all your fears and doubts...,9c6dc41d5ccd9968d07f055da5d8f741
3,e2c8830fbc86e5964478243099eec23a,You're ashamed about all your fears and doubts...,bfd04a73e9cffdf0e282c92219a86ea1
4,7624653ca8522ba93470843c74961b7d,"You showed him all the best of you,\nBut I'm a...",8d70930d09cd239c948408d1317d8659


In [13]:
meta = pd.DataFrame(meta_js)
meta.head()

,track_id,dttm,title,language,isrc,genres,duration
0,c3b9d6a354ca008aa4518329aaa21380,1.639688e+12,Happy New Year,EN,RUB422103970,"[""DANCE""]",161120.0
1,c57e3d13bbbf5322584a7e92e6f1f7ff,1.637762e+12,Bad Habits,EN,QZN882178276,"[""ELECTRONICS""]",362260.0
2,955f2aafe8717908c140bf122ba4172d,1.637768e+12,Por Esa Loca Vanidad,None,QZNJZ2122549,"[""FOLK"", ""LATINFOLK""]",260000.0
3,fae5a077c9956045955dde02143bd8ff,1.637768e+12,Mil Lagrimas,None,QZNJZ2166033,"[""FOLK"", ""LATINFOLK""]",190000.0
4,6bede082154d34fc18d9a6744bc95bf5,1.637768e+12,Sexo Humo y Alcohol,None,QZNJZ2122551,"[""FOLK"", ""LATINFOLK""]",203000.0


In [14]:
# Объединяем по столбцу track_id
data = covers.merge(lyrics, on='track_id', how='left').merge(meta, on='track_id', how='left')
#data = data_covers.merge(data_lyrics, on='track_id', how='outer').merge(data_meta, on='track_id', how='outer')
#data = covers.merge(lyrics, on='track_id', how='outer').merge(meta, on='track_id', how='outer')

data['dttm'] = pd.to_datetime(data['dttm'], unit='ms')
data['dttm'] = pd.to_datetime(data['dttm']).dt.strftime('%d-%m-%Y')#data['dttm'] = pd.to_datetime(data['dttm'], format='%d-%m-%Y')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72571 entries, 0 to 72570
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_track_id  5378 non-null   object 
 1   track_id           72571 non-null  object 
 2   track_remake_type  72571 non-null  object 
 3   lyricId            11097 non-null  object 
 4   text               11097 non-null  object 
 5   dttm               72571 non-null  object 
 6   title              72571 non-null  object 
 7   language           22598 non-null  object 
 8   isrc               72242 non-null  object 
 9   genres             72571 non-null  object 
 10  duration           72571 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.6+ MB


In [16]:
#напишем функцию для исследования данных
def research(data, name, figsize, silent):
    print(f'Размер данных:      {data.shape}')
    print(f'Количество явных дубликатов: {data.duplicated().sum()}')
    print(f'Наличие пропусков:           {data.isna().sum().sum()}')
    print('Пропущенные данные (в процентном соотношении):')
    logging.getLogger('matplotlib.font_manager').disabled = True
    print(round(data.isna().mean()*100).sort_values(ascending=False).head(15))
    display(data.head(3))
    if not silent:
        print(f'\nПроверка структуры {name}:')
        data.hist(linewidth=2, histtype='step', figsize=figsize)
        plt.suptitle(f'Гистограмма распеределения {name}', y=0.95, fontsize=12)
        plt.show()

        print()
        #тепловая карта корреляций
        plt.figure(figsize=(6, 6))
        matrix = np.triu(data.corr())
        heatmap = sns.heatmap(data.corr(), annot=True, fmt='.2g',
                          mask=matrix, square=True,
                          cmap='GnBu',  cbar=False,
                          xticklabels=True, yticklabels=True , vmin=0, vmax=1, center= 0)#'coolwarm', cmap=cmap,'Blues','BuGn'
        plt.suptitle(f'Тепловая карта матрицы корреляции {name}', y=0.90, fontsize=12)
        plt.show()
        print()
        display(data.describe())

In [17]:
research(data, 'общего датасета', figsize=(13, 7), silent=True)

Размер данных:      (72571, 11)
Количество явных дубликатов: 0
Наличие пропусков:           240443
Пропущенные данные (в процентном соотношении):
original_track_id    93.0
lyricId              85.0
text                 85.0
language             69.0
track_id              0.0
track_remake_type     0.0
dttm                  0.0
title                 0.0
isrc                  0.0
genres                0.0
duration              0.0
dtype: float64


,original_track_id,track_id,track_remake_type,lyricId,text,dttm,title,language,isrc,genres,duration
0,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,260f21d9f48e8de874a6e844159ddf28,Left a good job in the city\nWorkin' for the m...,11-11-2009,Proud Mary,EN,USFI86900049,"[""ROCK"", ""ALLROCK""]",187220.0
1,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,f3331cf99637ee24559242d13d8cf259,Left a good job in the city\nWorkin' for the m...,11-11-2009,Proud Mary,EN,USFI86900049,"[""ROCK"", ""ALLROCK""]",187220.0
2,fe7ee8fc1959cc7214fa21c4840dff0a,fe7ee8fc1959cc7214fa21c4840dff0a,ORIGINAL,2498827bd11eca5846270487e4960080,Some folks are born made to wave the flag\nOoh...,11-11-2009,Fortunate Son,EN,USFI86900065,"[""ROCK"", ""ALLROCK""]",137780.0


In [18]:
#data.to_csv("data.csv", index=False)

**Выводы:**
Провели первичный анализ:
- Сразу перевела данные даты в формат даты.
- Количество явных дубликатов: 0
- Наличие пропусков - 240443. Пропущенные данные (в процентном соотношении):
  - original_track_id    93.0
  - lyricId              85.0
  - text                 85.0
  - language             69.0
- Поработаем с пропусками:
  - в тексте, сделаем предобработку,
  - language - можем достать из названия и текста песен.

-  В genres - уберем кавычки.

## **2. Исследовательский анализ:**
  - Предобработка текстов
  - Работа с аномалиями, пропусками и дубликатами.

#### **Заполним пропуски в 'language'**

In [19]:
%%time
tqdm.pandas()

def detect_language(text):
    try:
        if not pd.isna(text):
            return detect(text)
    except:
        pass
    return pd.NA

def detect_lang_fill(row):
    if pd.isna(row['language']):
        if pd.notna(row['text']):
            row['language'] = detect_language(row['text'])
        elif pd.notna(row['title']):
            row['language'] = detect_language(row['title'])

    return row['language']

data['language'] = data.progress_apply(detect_lang_fill, axis=1)

100%|██████████| 72571/72571 [08:36<00:00, 140.55it/s]

CPU times: user 8min 10s, sys: 4.48 s, total: 8min 15s
Wall time: 8min 36s


In [20]:
data.tail(5)

,original_track_id,track_id,track_remake_type,lyricId,text,dttm,title,language,isrc,genres,duration
72566,4788e0bf61d80ef5ec9380aa8a8119d9,4788e0bf61d80ef5ec9380aa8a8119d9,ORIGINAL,NaN,NaN,28-09-2023,"Милый, прощай",ru,RUAGT2312928,"[""POP"", ""RUSPOP""]",178980.0
72567,None,78b2db35476f134dc3cdfbf4d77ba034,COVER,NaN,NaN,01-10-2023,Habits (Stay Hight),EN,TCAHK2396284,"[""ELECTRONICS""]",149570.0
72568,None,e720ff378efe032df56e0e656a6a92d3,COVER,NaN,NaN,05-10-2023,Arcade,EN,TCAHM2318975,"[""FOREIGNBARD"", ""BARD""]",201580.0
72569,554e33d79e258da91149c3a4985cf6a1,554e33d79e258da91149c3a4985cf6a1,ORIGINAL,NaN,NaN,05-10-2023,Май,bg,SMRUS0076417,"[""RUSRAP"", ""RAP""]",156870.0
72570,7b0f6ff24137be50cf5ea5f82d789448,7b0f6ff24137be50cf5ea5f82d789448,ORIGINAL,NaN,NaN,05-10-2023,Не улетай,ru,DGA0M2316512,"[""POP"", ""RUSPOP""]",148500.0


In [21]:
data['language'].value_counts() #.unique() #

EN    15866
en    13991
pt    10818
es     6728
de     2249
      ...  
YO        1
SA        1
MN        1
te        1
TG        1
Name: language, Length: 134, dtype: int64

In [22]:
#data_lang = data.copy()
#data_lang.to_csv("data_lang.csv", index=False)

#### **Сделаем предобработку текстов**

In [23]:
data['text'][0]

"Left a good job in the city\nWorkin' for the man ev'ry night and day\nAnd I never lost one minute of sleepin'\nWorryin' 'bout the way things might have been\n\nBig wheel keep on turnin'\nProud Mary keep on burnin'\nRollin', rollin', rollin' on the river\n\nCleaned a lot of plates in Memphis\nPumped a lot of 'pane down in New Orleans\nBut I never saw the good side of a city\n'Til I hitched a ride on a riverboat queen\n\nBig wheel keep on turnin'\nProud Mary keep on burnin'\nRollin', rollin', rollin' on the river\n\nRollin', rollin', rollin' on the river\n\nIf you come down to the river\nBet you gonna find some people who live\nYou don't have to worry 'cause you have no money\nPeople on the river are happy to give\n\nBig wheel keep on turnin'\nProud Mary keep on burnin'\nRollin', rollin', rollin' on the river\nRollin', rollin', rollin' on the river\nRollin', rollin', rollin' on the river\nRollin', rollin', rollin' on"

In [24]:
data['text'][1]

"Left a good job in the city\nWorkin' for the man every night and day\nAnd I never lost one minute of sleepin'\nWorryin' 'bout the way things might have been\n\nBig wheel keep on turnin'\nProud Mary keep on burnin'\nRollin', rollin'\nRollin' on the river\n\nCleaned a lot of plates in Memphis\nPumped a lot of 'pane down in New Orleans\nBut I never saw the good side of the city\n'Til I hitched a ride on a river boat queen\n\nBig wheel keep on turnin'\nProud Mary keep on burnin'\nRollin', rollin' (rollin')\nRollin' on the river\n\nRollin', rollin'\nRollin' on the river\n\nIf you come down to the river\nBet you gonna find some people who live\nYou don't have to worry 'cause you have no money\nPeople on the river are happy to give\n\nBig wheel keep on turnin'\nProud Mary keep on burnin'\nRollin', rollin'\nRollin' on the river\n\nRollin', rollin' (roll, Lord)\nRollin' on the river\nRollin', rollin'\nRollin' on the river\nRollin', rollin'\nRollin' on the river"

In [25]:
data['text'][2]

'Some folks are born made to wave the flag\nOoh, they\'re red, white and blue\nAnd when the band plays "Hail to the Chief"\nOoh, they point the cannon at you, Lord\n\nIt ain\'t me, it ain\'t me\nI ain\'t no senator\'s son, son\nIt ain\'t me, it ain\'t me\nI ain\'t no fortunate one, no\n\nSome folks are born silver spoon in hand\nLord, don\'t they help themselves, Lord?\nBut when the taxman come to the door\nLord, the house lookin\' like a rummage sale, yeah\n\nIt ain\'t me, it ain\'t me\nI ain\'t no millionaire\'s son, no, no\nIt ain\'t me, it ain\'t me\nI ain\'t no fortunate one, no\n\nYeah-yeah, some folks inherit star-spangled eyes\nOoh, they send you down to war, Lord\nAnd when you ask \'em, "How much should we give?"\nHoo, they only answer, "More, more, more, more"\n\nIt ain\'t me, it ain\'t me\nI ain\'t no military son, son, Lord\nIt ain\'t me, it ain\'t me\nI ain\'t no fortunate one, one\n\nIt ain\'t me, it ain\'t me\nI ain\'t no fortunate one, no, no, no\nIt ain\'t me, it ain\'

In [26]:
#приведем тексты к нижнему регистру
data['text'] = data['text'].str.lower()
data['title'] = data['title'].str.lower()
data['genres'] = data['genres'].str.lower()
data['language'] = data['language'].str.lower()

# убираем сокращенные формы глаголов в 'text'
data['text'] = data['text'].astype(str)
data['text'] = data['text'].apply(lambda x: contractions.fix(x))
#data['text'] = data['text'].str.replace("ev'ry", 'every')

# уберем ненужные символы
#data['text'] = data['text'].apply(lambda x: re.sub(r'\n', ' ', x))
data['text'] = data['text'].apply(lambda x: re.sub(r"ev'ry", 'every', x))
data['genres'] = [re.sub(r'"', '', genre) for genre in data['genres']]
data.head(3)

,original_track_id,track_id,track_remake_type,lyricId,text,dttm,title,language,isrc,genres,duration
0,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,260f21d9f48e8de874a6e844159ddf28,left a good job in the city\nworkin' for the m...,11-11-2009,proud mary,en,USFI86900049,"[rock, allrock]",187220.0
1,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,f3331cf99637ee24559242d13d8cf259,left a good job in the city\nworkin' for the m...,11-11-2009,proud mary,en,USFI86900049,"[rock, allrock]",187220.0
2,fe7ee8fc1959cc7214fa21c4840dff0a,fe7ee8fc1959cc7214fa21c4840dff0a,ORIGINAL,2498827bd11eca5846270487e4960080,some folks are born made to wave the flag\nooh...,11-11-2009,fortunate son,en,USFI86900065,"[rock, allrock]",137780.0


In [27]:
%%time
# Преобразуем текст в список токенов и удалим символы \n, оставим знаки препинания
data['text'] = data['text'].apply(lambda x: ' '.join([token for token in simple_preprocess(x) if token != '\n' or token.is_punct]))

CPU times: user 3.18 s, sys: 17 ms, total: 3.2 s
Wall time: 3.25 s


In [28]:
data['text'][0]

'left good job in the city workin for the man every night and day and never lost one minute of sleepin worryin bout the way things might have been big wheel keep on turnin proud mary keep on burnin rollin rollin rollin on the river cleaned lot of plates in memphis pumped lot of pane down in new orleans but never saw the good side of city til hitched ride on riverboat queen big wheel keep on turnin proud mary keep on burnin rollin rollin rollin on the river rollin rollin rollin on the river if you come down to the river bet you going to find some people who live you do not have to worry because you have no money people on the river are happy to give big wheel keep on turnin proud mary keep on burnin rollin rollin rollin on the river rollin rollin rollin on the river rollin rollin rollin on the river rollin rollin rollin on'

In [29]:
data['text'][1]

'left good job in the city workin for the man every night and day and never lost one minute of sleepin worryin bout the way things might have been big wheel keep on turnin proud mary keep on burnin rollin rollin rollin on the river cleaned lot of plates in memphis pumped lot of pane down in new orleans but never saw the good side of the city til hitched ride on river boat queen big wheel keep on turnin proud mary keep on burnin rollin rollin rollin rollin on the river rollin rollin rollin on the river if you come down to the river bet you going to find some people who live you do not have to worry because you have no money people on the river are happy to give big wheel keep on turnin proud mary keep on burnin rollin rollin rollin on the river rollin rollin roll lord rollin on the river rollin rollin rollin on the river rollin rollin rollin on the river'

In [30]:
data['text'][2]

'some folks are born made to wave the flag ooh they are red white and blue and when the band plays hail to the chief ooh they point the cannon at you lord it are not me it are not me are not no senator son son it are not me it are not me are not no fortunate one no some folks are born silver spoon in hand lord do not they help themselves lord but when the taxman come to the door lord the house lookin like rummage sale yeah it are not me it are not me are not no millionaire son no no it are not me it are not me are not no fortunate one no yeah yeah some folks inherit star spangled eyes ooh they send you down to war lord and when you ask them how much should we give hoo they only answer more more more more it are not me it are not me are not no military son son lord it are not me it are not me are not no fortunate one one it are not me it are not me are not no fortunate one no no no it are not me it are not me are not no fortunate son no no no it are not me it are not me'

In [31]:
data['text'][6]

'state of emergency yeah yeah yeah oh yeah oh oh oh yeah yeah yeah oh yeah oh oh oh remember the time baby yeah yeah yeah oh yeah oh oh oh are not got no money are not got no car to take you on date cannot even buy you flowers but together we will be the perfect soulmates talk to me girl oh baby it is alright now you are not got to flaunt for me if we go dutch you can still touch my love it is free we can work without the perks just you and me thug it out til we get it right baby if you strip you can get tip because like you just the way you are am about to strip and am well equipped can you handle me the way are do not need the or the car keys boy like you just the way you are let me see ya strip you can get tip because like like like are not got no visa are not got no red american express we cannot go nowhere exotic it do not matter because am the one that loves you best talk to me girl oh baby it is alright now you are not got to flaunt for me if we go dutch you can still touch my l

#### **Проверим на наличие неявных дубликатов**

In [32]:
#Посчитаем неявные дубликаты (отличаются только по lyricId)
duplicate_count = data.drop('lyricId', axis=1).duplicated().sum()
print('Количество неявных дубликатов', duplicate_count)
duplicates = data[data.drop('lyricId', axis=1).duplicated()]
display(duplicates['text'].head(2))

Количество неявных дубликатов 281


7     in state of emergency are not got no money are...
34    see trees of green red roses too see them bloo...
Name: text, dtype: object

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72571 entries, 0 to 72570
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_track_id  5378 non-null   object 
 1   track_id           72571 non-null  object 
 2   track_remake_type  72571 non-null  object 
 3   lyricId            11097 non-null  object 
 4   text               72571 non-null  object 
 5   dttm               72571 non-null  object 
 6   title              72571 non-null  object 
 7   language           72510 non-null  object 
 8   isrc               72242 non-null  object 
 9   genres             72571 non-null  object 
 10  duration           72571 non-null  float64
dtypes: float64(1), object(10)
memory usage: 8.7+ MB


In [34]:
# Удаляем неявные дубикаты
#data1 = data[~data.isin(duplicates)].dropna()
data = data.drop_duplicates(subset=data.columns.difference(['lyricId'])).reset_index(drop=True)
#Проверяем
duplicate_count = data.drop('lyricId', axis=1).duplicated().sum()
print('Количество неявных дубликатов после обработки:', duplicate_count)

Количество неявных дубликатов после обработки: 0


In [35]:
#data[data['track_id'].duplicated()] #duplicate_count1 =
data['track_id'].duplicated().value_counts()

False    71597
True       693
Name: track_id, dtype: int64

In [36]:
data[data['track_id'] == '2bfb9427a1d97d16ab61ff31d6408870']

,original_track_id,track_id,track_remake_type,lyricId,text,dttm,title,language,isrc,genres,duration
72019,None,2bfb9427a1d97d16ab61ff31d6408870,COVER,3525530bd73b2802420fd85c265ff6ab,хочу запомнить как смята постель как ты одевае...,15-06-2023,спектакль окончен,ru,AEA2Z2314296,"[punk, postpunk]",189480.0


In [37]:
print('Количество неявных дубликатов по "track_id":', len(data[data.duplicated(subset=['track_id', 'track_remake_type'], keep='first')]))
#data.drop(duplicates.index, inplace=True)

Количество неявных дубликатов по "track_id": 693


In [38]:
#data.drop_duplicates(subset=['track_id', 'track_remake_type'], keep='first', inplace=True).reset_index(drop=True)
#data = data.reset_index(drop=True)
#print(f'Количество неявных дубликатов по "track_id" после обработки:', len(data[data.duplicated(subset=['track_id', 'track_remake_type'], keep='first')]))

In [39]:
#разделим список в genre и создадим новые строки с одним его значением
#from ast import literal_eval
#data['genres'] = data['genres'].apply(literal_eval)
#data['genres_list'] = data['genres'].apply(lambda x: x if isinstance(x, list) else [])
#data1 = data.explode('genres')

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72290 entries, 0 to 72289
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_track_id  5253 non-null   object 
 1   track_id           72290 non-null  object 
 2   track_remake_type  72290 non-null  object 
 3   lyricId            10816 non-null  object 
 4   text               72290 non-null  object 
 5   dttm               72290 non-null  object 
 6   title              72290 non-null  object 
 7   language           72229 non-null  object 
 8   isrc               71965 non-null  object 
 9   genres             72290 non-null  object 
 10  duration           72290 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.1+ MB


### **Заполним пропуски**

In [41]:
data['language'] = data['language'].fillna(value='No')
data['title'] = data['title'].fillna(value='No')
data['duration'] = data['duration'].fillna(value=0)

In [42]:
research(data, 'общего датасета', figsize=(13, 7), silent=True)

Размер данных:      (72290, 11)
Количество явных дубликатов: 0
Наличие пропусков:           128836
Пропущенные данные (в процентном соотношении):
original_track_id    93.0
lyricId              85.0
track_id              0.0
track_remake_type     0.0
text                  0.0
dttm                  0.0
title                 0.0
language              0.0
isrc                  0.0
genres                0.0
duration              0.0
dtype: float64


,original_track_id,track_id,track_remake_type,lyricId,text,dttm,title,language,isrc,genres,duration
0,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,260f21d9f48e8de874a6e844159ddf28,left good job in the city workin for the man e...,11-11-2009,proud mary,en,USFI86900049,"[rock, allrock]",187220.0
1,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,f3331cf99637ee24559242d13d8cf259,left good job in the city workin for the man e...,11-11-2009,proud mary,en,USFI86900049,"[rock, allrock]",187220.0
2,fe7ee8fc1959cc7214fa21c4840dff0a,fe7ee8fc1959cc7214fa21c4840dff0a,ORIGINAL,2498827bd11eca5846270487e4960080,some folks are born made to wave the flag ooh ...,11-11-2009,fortunate son,en,USFI86900065,"[rock, allrock]",137780.0


In [43]:
#data_final = data.copy()
#data_final.to_csv("data_final.csv", index=False)

#### Вывод:
- проведена обработка текстов:
  - Зменили сокращенную форму глагола в `text` на полную;
  - убрали ненужные символы;
  - привели к нижнему регистру;
- Проведена проверка на наличие неявных дубликатов по `track_id`, а также без учета `text` , удалили их:
  > Обнаружено 974 неявных дубликатов, удалили их.

- Провели токенизацию текста с песнями `text`.
- заполнили пропуски:
  -  language из названия и текста песни, оставшиеся на 'No'
  - 'title' на 'No'
  - 'duration' на 0


In [44]:
#проверка орфографии
#!pip install textblob >>N
#from textblob import TextBlob
#textBlb = TextBlob(data['text'][0])            # Making our first textblob
#textCorrected = textBlb.correct()

### **3. Формирование признаков:**
- Анализ данных,
- Удаление неинформативных признаков, генерация ряда признаков, по необходимости.

#### **Создаем эмбендинговую  модель**
Не хватило времени ее проверить - не успели создать эмбендинги

In [45]:
%%time
# Загрузка модели SentenceTransformer
model = SentenceTransformer("ai-forever/FRED-T5-1.7B")
def embeddings(text):
    return model.encode(text)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 7.5 s, sys: 7.81 s, total: 15.3 s
Wall time: 46.4 s


In [46]:
#создадим небольшой датасет для проверки работоспособности эмбендинговой модели
data_cut = data.head(10)

In [47]:
%%time

tqdm.pandas()
# Создаем эмбеддинги для столбца 'title'
data_cut['title_embeddings'] = data_cut['title'].progress_apply(embeddings)
data_emb = data.copy()
data_emb.to_csv("data_emb.csv", index=False)

100%|██████████| 10/10 [00:04<00:00,  2.01it/s]


CPU times: user 5.3 s, sys: 56.2 ms, total: 5.35 s
Wall time: 5.7 s


In [48]:
%%time
tqdm.pandas()
# Создаем эмбеддинги для столбца 'genre'
data_cut['genre_embedding'] = data_cut['genres'].progress_apply(embeddings)
data_emb = data_cut.copy()
data_emb.to_csv("data_emb.csv", index=False)

100%|██████████| 10/10 [00:05<00:00,  1.75it/s]

CPU times: user 5.62 s, sys: 27.7 ms, total: 5.65 s
Wall time: 5.75 s


In [49]:
%%time
tqdm.pandas()
# Создаем объединенный эмбеддинг для столбцов 'genre' и 'track_id'
data_cut['genre_track_id_embedding'] = data_cut.progress_apply(lambda row: model.encode(row['genres'] + " " + row['track_id']), axis=1)

# Вывести результат
data_emb1 = data_cut.copy()
data_emb1.to_csv("data_emb1.csv", index=False)

100%|██████████| 10/10 [00:09<00:00,  1.11it/s]

CPU times: user 9.02 s, sys: 28.1 ms, total: 9.04 s
Wall time: 9.15 s


In [50]:
display(data_cut)

,original_track_id,track_id,track_remake_type,lyricId,text,dttm,title,language,isrc,genres,duration,title_embeddings,genre_embedding,genre_track_id_embedding
0,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,260f21d9f48e8de874a6e844159ddf28,left good job in the city workin for the man e...,11-11-2009,proud mary,en,USFI86900049,"[rock, allrock]",187220.0,"[0.0011266887, -0.0048668757, -0.00245695, -0....","[0.0013713717, 0.0013830372, -0.0030863965, -0...","[0.006809474, -0.0025031776, -0.005508644, 0.0..."
1,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,f3331cf99637ee24559242d13d8cf259,left good job in the city workin for the man e...,11-11-2009,proud mary,en,USFI86900049,"[rock, allrock]",187220.0,"[0.0011266887, -0.0048668757, -0.00245695, -0....","[0.0013713717, 0.0013830372, -0.0030863965, -0...","[0.006809474, -0.0025031776, -0.005508644, 0.0..."
2,fe7ee8fc1959cc7214fa21c4840dff0a,fe7ee8fc1959cc7214fa21c4840dff0a,ORIGINAL,2498827bd11eca5846270487e4960080,some folks are born made to wave the flag ooh ...,11-11-2009,fortunate son,en,USFI86900065,"[rock, allrock]",137780.0,"[0.009949996, -0.0015443905, -0.006491226, -0....","[0.0013713717, 0.0013830372, -0.0030863965, -0...","[0.008026072, -0.0034410367, -0.0005314559, -0..."
3,cd89fef7ffdd490db800357f47722b20,cd89fef7ffdd490db800357f47722b20,ORIGINAL,5237001311d4062bf2b80de30652bf58,uno por pobre feo hombre pero antoja ay ome te...,21-09-2009,la camisa negra,es,USUL10400965,"[pop, folk, latinfolk]",216840.0,"[0.0036356114, -0.0011110839, -0.0058290297, -...","[-0.002829403, -0.000318735, -0.0030060709, -0...","[0.002671208, -0.0029204066, -0.0038161918, 8...."
4,995665640dc319973d3173a74a03860c,995665640dc319973d3173a74a03860c,ORIGINAL,e5b1b57090b728e8d98d2b4d9b781bf4,yeah yeah remember the time baby yeah are not ...,16-11-2009,the way i are,en,USUM70722806,"[foreignrap, rap]",179660.0,"[0.00280195, 0.0003117473, -0.00030422676, -0....","[0.00563304, -0.0022621434, -0.002719903, -0.0...","[0.005826182, 0.0011910264, -0.0025019941, -0...."
5,995665640dc319973d3173a74a03860c,995665640dc319973d3173a74a03860c,ORIGINAL,b6625d84706fefe8782e63bd36067bc2,in state of emergency are not got no money are...,16-11-2009,the way i are,en,USUM70722806,"[foreignrap, rap]",179660.0,"[0.00280195, 0.0003117473, -0.00030422676, -0....","[0.00563304, -0.0022621434, -0.002719903, -0.0...","[0.005826182, 0.0011910264, -0.0025019941, -0...."
6,995665640dc319973d3173a74a03860c,995665640dc319973d3173a74a03860c,ORIGINAL,4b30eb13f54a1d83f34202ab8e8a3357,state of emergency yeah yeah yeah oh yeah oh o...,16-11-2009,the way i are,en,USUM70722806,"[foreignrap, rap]",179660.0,"[0.00280195, 0.0003117473, -0.00030422676, -0....","[0.00563304, -0.0022621434, -0.002719903, -0.0...","[0.005826182, 0.0011910264, -0.0025019941, -0...."
7,None,d6288499d0083cc34e60a077b7c4b3e1,COVER,NaN,nan,17-09-2009,extraball,en,FR8Q10900116,[electronics],212620.0,"[-0.0019890147, -0.0045787725, -0.007676913, -...","[0.004979204, -0.00064526557, -0.00030279983, ...","[0.002797134, -0.0046878075, -0.0041918363, -0..."
8,None,4da9d7b6d119db4d2d564a2197798380,COVER,58b6145f2fb180f8cdc2067b4f1baebd,cannot buy me love cannot buy me love cannot b...,17-09-2009,can't buy me love,en,USGR10110569,"[jazz, vocaljazz]",158950.0,"[0.0021620712, -0.00882596, -0.008821242, 0.00...","[-0.0027748537, 0.0009906983, -0.004430055, -0...","[-0.0026662438, 0.001698072, -0.0043553836, 0...."
9,None,2bf283c05b601f21364d052ca0ec798d,COVER,eb38211a25c1320991c5a23ad2417f33,wednesday morning at five of the clock as the ...,17-09-2009,she's leaving home,en,USGR19900418,[jazz],356070.0,"[-0.00091484876, -0.0015381086, -0.0061616576,...","[-0.0052074282, 0.000329711, -0.0013125452, 0....","[0.00040179098, -0.0045607723, -0.0024972374, ..."


**Вывод:**
- Модель рабочая - все проверено.

#### **Удаление неинформативных признаков**

In [51]:
def get_data_info(data):
    display(data.sample(5))
    display(data.info())
    display(data.describe(include='all'))

In [52]:
corpus = data[['title', 'language', 'duration', 'track_remake_type']]\
                      .drop_duplicates()\
                      .reset_index(drop=True)

get_data_info(corpus)

,title,language,duration,track_remake_type
25885,buona sera signorina,it,192660.0,COVER
4880,international love,en,226240.0,COVER
4145,e la vita la vita,it,223220.0,COVER
23362,bellyache,de,175800.0,COVER
37660,"daredevil (from ""persona 5 strikers"")",en,290690.0,COVER


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69975 entries, 0 to 69974
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              69975 non-null  object 
 1   language           69975 non-null  object 
 2   duration           69975 non-null  float64
 3   track_remake_type  69975 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.1+ MB


None

,title,language,duration,track_remake_type
count,69975,69975,6.997500e+04,69975
unique,43984,93,NaN,2
top,smooth criminal,en,NaN,COVER
freq,84,28136,NaN,65707
mean,NaN,NaN,2.050657e+05,NaN
std,NaN,NaN,8.588166e+04,NaN
min,NaN,NaN,0.000000e+00,NaN
25%,NaN,NaN,1.612250e+05,NaN
50%,NaN,NaN,1.992500e+05,NaN
75%,NaN,NaN,2.391505e+05,NaN


In [53]:
corpus['track_remake_type'].unique()

array(['ORIGINAL', 'COVER'], dtype=object)

In [54]:
def codirovanie(text):
    if text == 'ORIGINAL':
        return 1
    else:
        return 0

In [55]:
corpus['type'] = corpus['track_remake_type'].apply(codirovanie)

corpus = corpus.drop(['track_remake_type'], axis=1)

target = corpus['type']
features = corpus.drop(['type'], axis=1)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE,stratify=corpus['type']
)

In [57]:
print('X_train.shape = ', X_train.shape, 'y_train.shape = ', y_train.shape)
print('X_test.shape = ', X_test.shape, 'y_test.shape = ', y_test.shape)

X_train.shape =  (52481, 3) y_train.shape =  (52481,)
X_test.shape =  (17494, 3) y_test.shape =  (17494,)


In [58]:
get_data_info(X_train)

,title,language,duration
14051,love myself,no,146510.0
15617,nothing's gonna change my love for you,en,246420.0
48509,"не верь, не бойся",ru,176970.0
40692,a galera não vai sair,pt,150770.0
64117,rita,it,158900.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 52481 entries, 33758 to 47995
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     52481 non-null  object 
 1   language  52481 non-null  object 
 2   duration  52481 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.6+ MB


None

,title,language,duration
count,52481,52481,5.248100e+04
unique,34809,89,NaN
top,smooth criminal,en,NaN
freq,60,21117,NaN
mean,NaN,NaN,2.051258e+05
std,NaN,NaN,8.580118e+04
min,NaN,NaN,0.000000e+00
25%,NaN,NaN,1.610400e+05
50%,NaN,NaN,1.991300e+05
75%,NaN,NaN,2.390900e+05


In [59]:
get_data_info(X_test)

,title,language,duration
26356,สาวกกกอก,th,280510.0
11309,ещё хочу,ru,205450.0
65231,te necesito,it,197830.0
12184,gregson: 2.3 courante,en,99610.0
58877,contatinho reserva,pt,153430.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17494 entries, 14205 to 44600
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     17494 non-null  object 
 1   language  17494 non-null  object 
 2   duration  17494 non-null  float64
dtypes: float64(1), object(2)
memory usage: 546.7+ KB


None

,title,language,duration
count,17494,17494,1.749400e+04
unique,13856,71,NaN
top,smooth criminal,en,NaN
freq,24,7019,NaN
mean,NaN,NaN,2.048853e+05
std,NaN,NaN,8.612485e+04
min,NaN,NaN,0.000000e+00
25%,NaN,NaN,1.618700e+05
50%,NaN,NaN,1.995900e+05
75%,NaN,NaN,2.392775e+05


In [60]:
#категориальные признаки
cat_features = X_train.select_dtypes(include='object').columns.to_list()
cat_features

['title', 'language']

In [61]:
num_features = X_train.select_dtypes(exclude='object').columns.to_list()
num_features

['duration']

In [76]:
X_train_oe = X_train.copy()
X_test_oe = X_test.copy()

In [78]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(X_train_oe[cat_features])
X_train_oe[cat_features] = encoder.transform(X_train_oe[cat_features])
X_test_oe[cat_features] = encoder.transform(X_test_oe[cat_features])

In [79]:
scaler=StandardScaler()
scaler.fit(X_train_oe[num_features])

X_train_oe[num_features]=scaler.transform(X_train_oe[num_features])
X_test_oe[num_features]=scaler.transform(X_test_oe[num_features])

# RandomForestClassifier

In [80]:
model_rf = RandomForestClassifier()

param_grid = {
    'max_depth': [None] + [i for i in range(2, 7)],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [2, 10],
    'n_estimators': [10, 50, 100],
}




grid_search_rf = GridSearchCV(model_rf, param_grid, cv=5,scoring='roc_auc')


grid_search_rf.fit(X_train_oe, y_train)
grid_search_rf.best_params_
grid_search_rf.best_score_


0.9199553629796368

# CatBoostClassifier

In [81]:
model_cat =  CatBoostClassifier(random_state=RANDOM_STATE, verbose=0)
param_grid = {
    'learning_rate': [0.1, 0.3],
    'iterations': [50, 100],
    'l2_leaf_reg': [3, 9]
}


grid_search_cat = GridSearchCV(model_cat, param_grid, cv=5,scoring='roc_auc')


grid_search_cat.fit(X_train_oe, y_train)
grid_search_cat.best_params_
grid_search_cat.best_score_

0.9274416520625162

# XGBClassifier

In [83]:
model_xgb =  XGBClassifier(random_state=RANDOM_STATE, verbose=0)

param_grid = {
    'n_estimators': [10, 50, 100],
    'learning_rate': [0.1, 0.3],
    'max_depth': [None] + [i for i in range(2, 7)],

}

grid_search_xgb = GridSearchCV(model_xgb, param_grid, cv=5,scoring='roc_auc')


grid_search_xgb.fit(X_train_oe, y_train)
grid_search_xgb.best_params_
grid_search_xgb.best_score_

0.9280860788193197

In [86]:
result = pd.DataFrame(
    [grid_search_rf.best_score_, grid_search_cat.best_score_, grid_search_xgb.best_score_],
    index=['RandomForestClassifier', 'CatBoostClassifier', 'XGBClassifier'],
    columns=['roc_auc']
)
result

,roc_auc
RandomForestClassifier,0.919955
CatBoostClassifier,0.927442
XGBClassifier,0.928086


# Тестирование

In [91]:
clf = XGBClassifier(**grid_search_xgb.best_params_ )

model_xgb = clf.fit(X_train_oe, y_train)

In [92]:
# Проверим на тестовой выборке качество модели 'XGBClassifier'


xgb_predict = model_xgb.predict(X_test_oe)
prediction_xgb = model_xgb.predict_proba(X_test_oe)[:,1]

roc_test = roc_auc_score(y_test, prediction_xgb)
accuracy_test = accuracy_score(y_test, xgb_predict)
f1_test = f1_score(y_test, xgb_predict)

print("roc_auc_score_test:", roc_test)
print("accuracy_score_test:", accuracy_test)
print("f1_score_test:", f1_test)

roc_auc_score_test: 0.917576949599914
accuracy_score_test: 0.9549559849091117
f1_score_test: 0.5612472160356349
